In [1]:
import pandas as pd
import numpy as np
import chromedriver_autoinstaller
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [14]:
url = lambda x: f"https://www.cybo.com/UA/kiev-kiev/hotels-&-travel/?p={x}"
            
chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  #크롬드라이버 버전 확인
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito") # 시크릿 모드
chrome_options.add_argument("--no-sandbox") # 리소스에 대한 엑서스 방지
chrome_options.add_argument("--disable-setuid-sandbox") # 크롬 충돌 방지
chrome_options.add_argument("--disable-dev-shm-usage") # 메모리 부족 에러 방지
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
try: # 크롬 드라이버
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)   
except:
    chromedriver_autoinstaller.install(True)
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        # WebDruverException Error 방지 기존의 드라이버 버젼으로 지정
        # driver = webdriver.Chrome(executable_path='/Users/cmblir/Python/Musinsa-Analysis/100/chromedriver')


In [32]:
link_lst = []
map_lst = []
for idx in range(1, 11):
    driver.get(url(idx))
    div_lst = driver.find_element(By.XPATH, '//*[@id="bottom"]')
    href_lst = div_lst.find_elements(By.TAG_NAME, 'a')
    for href in href_lst:
        link = href.get_attribute("href")
        if "UA-biz" in link and link not in link_lst: link_lst.append(link)
        elif "r/biz" in link and link not in map_lst: map_lst.append(link)

In [33]:
len(link_lst)

100

In [ ]:
company_df = pd.DataFrame()
for link in tqdm(link_lst):
    driver.get(link)
    company_name = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div/div/div/h1').text
    company_address = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/div/div/div/div[2]').text
    left_content = driver.find_element(By.XPATH, '//*[@id="left-contact"]').text
    lst_split = left_content.split("\n")
    for idx in range(len(lst_split)):
        if lst_split[idx-1] == "Phone": phone_number = lst_split[idx]
        elif lst_split[idx-1] == "Website ": company_url = lst_split[idx]
    about = driver.find_element(By.XPATH, '//*[@id="right_col"]/div[2]/div/div/div[1]/div').text
    ISIC_code = driver.find_element(By.XPATH, '//*[@id="right_col"]/div[2]/div/div/div[2]/div[2]/span[2]/span').text
    categories = driver.find_element(By.XPATH, '//*[@id="right_col"]/div[2]/div/div/div[2]/div[1]').text
    company_dict = {"company_name" : company_name,
                "company_address" : company_address,
                "phone_number" : phone_number,
                "company_url" : company_url,
                "about" : about,
                "ISIC_code" : ISIC_code,
                "categories" : categories}
    if company_df.empty : company_df = pd.DataFrame(company_dict, index=[0])
    else: company_df = company_df.append(company_dict, ignore_index=True)

In [36]:
company_df.to_csv("우크라이나.csv", index=False)